## Exercise 04 : A/B-testing

In [105]:
import pandas as pd
import sqlite3

## create a connection to the database using the library sqlite3
## using only one query for each of the groups, create two dataframes: test_results and control_results with the columns time and avg_diff and only two rows

- time should have the values: after and before
- avg_diff contains the average delta among all the users for the time period
before each of them made their first visit to the page and afterward
- only take into account the users that have observations before and after
- we still are not using the lab ’project1’

In [108]:
try:
  connect = sqlite3.connect('data/checking-logs.sqlite')
  print('OK. Connected to sqlite')

except sqlite3.Error as error:
  print('Error with connection:', error)

OK. Connected to sqlite


In [110]:
query = '''
WITH user_deltas AS (
    SELECT 
        t.uid,
        t.first_commit_ts,
        t.first_view_ts,
        d.deadlines,
        CAST((JulianDay(d.deadlines, 'unixepoch') - JulianDay(t.first_commit_ts)) * 24 AS INT) AS delta_hours
    FROM test t
    LEFT JOIN deadlines d ON t.labname = d.labs
    WHERE t.labname != 'project1'
),
users_with_both_times AS (
    SELECT uid
    FROM (
        SELECT 
            uid,
            CASE WHEN first_commit_ts < first_view_ts THEN 'before'
                 ELSE 'after'
            END as time
        FROM test
        LEFT JOIN deadlines ON test.labname=deadlines.labs
        WHERE labname <> 'project1'
    ) t
    GROUP BY uid
    HAVING COUNT(DISTINCT time)=2
)

SELECT 
    CASE 
        WHEN t.first_commit_ts < t.first_view_ts THEN 'before'
        ELSE 'after'
    END AS time,
    AVG(t.delta_hours) AS avg_diff
FROM user_deltas t
JOIN users_with_both_times u ON t.uid = u.uid
GROUP BY time;
'''

test_results = pd.io.sql.read_sql(query, connect)
test_results

,time,avg_diff
0,after,104.6000
1,before,60.5625


In [112]:
query = '''
WITH user_deltas AS (
    SELECT 
        c.uid,
        c.first_commit_ts,
        c.first_view_ts,
        d.deadlines,
        CAST((JulianDay(d.deadlines, 'unixepoch') - JulianDay(c.first_commit_ts)) * 24 AS INT) AS delta_hours
    FROM control c
    LEFT JOIN deadlines d ON c.labname = d.labs
    WHERE c.labname != 'project1'
),
users_with_both_times AS (
    SELECT uid
    FROM (
        SELECT 
            uid,
            CASE WHEN first_commit_ts < first_view_ts THEN 'before'
                 ELSE 'after'
            END as time
        FROM control
        LEFT JOIN deadlines ON control.labname=deadlines.labs
        WHERE labname <> 'project1'
    ) c
    GROUP BY uid
    HAVING COUNT(DISTINCT time)=2
)

SELECT 
    CASE 
        WHEN c.first_commit_ts < c.first_view_ts THEN 'before'
        ELSE 'after'
    END AS time,
    AVG(c.delta_hours) AS avg_diff
FROM user_deltas c
JOIN users_with_both_times u ON c.uid = u.uid
GROUP BY time;
'''
control_results = pd.io.sql.read_sql(query, connect)
control_results

,time,avg_diff
0,after,117.636364
1,before,99.464286


In [114]:
percent_test = round((test_results.avg_diff[0] - test_results.avg_diff[1]) / test_results.avg_diff[0] * 100, 0)
print(percent_test, '%')

42.0 %


In [116]:
percent_control = round((control_results.avg_diff[0] - control_results.avg_diff[1]) / control_results.avg_diff[0] * 100, 0)
print(percent_control, '%')

15.0 %


## В тестовой группе дельта до первого посещения новостной ленты значительно отличается по сравнению с дельтой после.
## Эта разница существенно меньше в контрольной группе.
## Гипотеза подтвердилась. Новостная лента положительно повлияла на поведение студентов: они начали работать над лабораторными работами раньше. В контрольной группе такого значительного эффекта не наблюдается, что подтверждает, что изменения связаны именно с новостной лентой. Создание страницы было удачной идеей, и ее можно внедрить для всей группы.